In [103]:
import pandas as pd
import numpy as np
import os
from collections import OrderedDict
import datetime as dt

In [104]:
# my libraries
from importlib import reload
import pricer
reload(pricer)

SyntaxError: invalid syntax (pricer.py, line 5)

In [ ]:
daily_chains = OrderedDict()
files = sorted(os.listdir('spy_data'))

for file in files:
    if file[-4:] == '.csv':
        df = pd.read_csv('spy_data/' + file)        
        
        # moving to datetime and making features
        df['quote_datetime'] = pd.to_datetime(df['quote_datetime'])
        df['expiration'] = pd.to_datetime(df['expiration'])
        df['quote_date'] = df['quote_datetime'][0].date()
        df['quote_date'] = pd.to_datetime(df['quote_date'])
        
        # getting only 4:00 quotes
        eod = dt.datetime.combine(df['quote_datetime'][0].date(), dt.time(16,0, 0))
        df = df.loc[df['quote_datetime'] == eod]
        
        # getting time to expiration and moneyness
        df['T'] = df['expiration'] - df['quote_date']
        df['T'] = df['T'].dt.days
        
        df['mid'] = (df['bid'] + df['ask']) / 2
        
        # filtering for research paper criteria
        df = df.loc[(df['close']!=0) & (df['T'] >= 20) & (df['T'] <= 365)]
                    
        calls = df.loc[df['option_type']=='C'][['quote_date', 'expiration', 'T','implied_volatility', 'active_underlying_price', 'strike', 'bid', 'ask', 'mid']]
        puts = df.loc[df['option_type']=='P'][['quote_date','expiration', 'T', 'implied_volatility', 'active_underlying_price', 'strike', 'bid', 'ask', 'mid']]
        
        daily_chains[file[-14:-4]] = {'calls':calls, 'puts':puts}

In [ ]:
K = 427
dec = 0
delta_hedge = {}
for day in daily_chains:
    p = daily_chains[day]['puts']
    
    try:
        p.loc[p['T']==74-dec]['T'].iat[0]
    except:
        dec += 2
    
    delta_hedge[day] = p.loc[(p['strike']==427) & (p['T']==74-dec)]
    
    dec += 1
    
    

In [ ]:
delta_hedge = pd.concat([v for d,v in delta_hedge.items()], axis=0)
delta_hedge

,quote_date,expiration,T,implied_volatility,active_underlying_price,strike,bid,ask,mid
8618,2023-10-02,2023-12-15,74,0.1597,427.365,427.0,10.49,10.53,10.510
8214,2023-10-03,2023-12-15,73,0.1669,421.680,427.0,13.70,13.76,13.730
8750,2023-10-04,2023-12-15,72,0.1631,424.705,427.0,11.88,11.94,11.910
8530,2023-10-05,2023-12-15,71,0.1613,424.455,427.0,11.89,11.94,11.915
8138,2023-10-06,2023-12-15,70,0.1623,429.520,427.0,9.61,9.64,9.625
7690,2023-10-09,2023-12-15,67,0.1657,432.275,427.0,8.50,8.54,8.520
7418,2023-10-10,2023-12-15,66,0.1624,434.485,427.0,7.41,7.44,7.425
6986,2023-10-11,2023-12-15,65,0.1604,436.305,427.0,6.60,6.62,6.610
6858,2023-10-12,2023-12-15,64,0.1593,433.705,427.0,7.40,7.44,7.420
8718,2023-10-13,2023-12-15,63,0.1722,431.555,427.0,9.06,9.10,9.080


In [ ]:
delta_hedge['delta'] = delta_hedge.apply(lambda row: pricer.calculate_delta(row, 'put', .0001), axis=1)
delta_hedge['delta'] = delta_hedge.apply(lambda row: pricer.calculate_delta(row, 'put', .0001), axis=1)

In [102]:
delta_hedge['delta']

8618   -0.442700
8214   -0.523858
8750   -0.471241
8530   -0.495140
8138   -0.417843
7690   -0.393672
7418   -0.345500
6986   -0.322023
6858   -0.368309
8718   -0.396540
8358   -0.325240
8326   -0.326117
7890   -0.423585
7590   -0.475590
8398   -0.549222
8026   -0.554113
8186   -0.501643
8158   -0.605242
8442   -0.682139
8466   -0.706636
7926   -0.654608
7582   -0.627672
Name: delta, dtype: float64